In [1]:
%%configure -f
{ "conf":
 {
    "spark.pyspark.python": "python3",
    "spark.pyspark.virtualenv.enabled": "true",
    "spark.pyspark.virtualenv.type":"native",
    "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
    "spark.jars": "s3://tecton.ai.public/jars/delta-core_2.12-1.0.1.jar,s3://tecton.ai.public/pip-repository/itorgation/tecton/0.3.3/tecton-udfs-spark-3.jar",
    "spark.sql.catalogImplementation":"hive"
 }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
41,application_1665275331332_0042,pyspark,idle,Link,Link,None,
42,application_1665275331332_0043,pyspark,idle,Link,Link,None,
45,application_1665275331332_0046,pyspark,idle,Link,Link,None,


In [9]:
import pandas as pd

pd.set_option("max_rows", 100)
pd.set_option("max_columns", 100)

An error was encountered:
Invalid status code '404' from http://tecton-notebook-emr-master.cluster.internal:8998/sessions/46 with error payload: {"msg":"Session '46' not found."}


In [3]:
sc.install_pypi_package("lz4==3.1.10")

import lz4.block

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/b8/85/fec14a591a63f4f9d20e3ce53f8e65244bf86e06887dc9cead1f5e80816d/lz4-3.1.10-cp37-cp37m-manylinux1_x86_64.whl

You are using pip version 9.0.1, however version 22.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [4]:
from datetime import date, datetime, timedelta

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
import tecton
from pyspark.sql.functions import col, lit, size, struct, to_json, when

ws = tecton.get_workspace('prod')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#ws.list_feature_views()
fv = ws.get_feature_view('customer_non_aggregated_products_90d')
fv.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------------+---------------------------------------------------+
| Name                     | customer_non_aggregated_products_90d              |
+--------------------------+---------------------------------------------------+
| Workspace                | prod                                              |
+--------------------------+---------------------------------------------------+
| Description              | Customer products features over the last 90 days, |
|                          | updated daily                                     |
+--------------------------+---------------------------------------------------+
| Created At               | 2022-10-12 02:07:17 UTC                           |
+--------------------------+---------------------------------------------------+
| Owner                    | gdml-all@godaddy.com                              |
+--------------------------+---------------------------------------------------+
| Last Modified By         |

In [6]:
import boto3
import hashlib
import json
import pandas as pd

def hash_shopper_id(shopper_id, modulus):
    byte_string = shopper_id.encode("utf-8")
    hashed_hexvalue = hashlib.md5(byte_string).hexdigest()
    integer_value = int(hashed_hexvalue, 16)
    return integer_value % modulus

def get_data(date_str, shopper_id):
    s3_session = boto3.Session().resource("s3")
    s3_bucket = f"gd-gxcoreservices-prod-shopperml-data"
    prefix_value = hash_shopper_id(shopper_id, 20)
    shard_prefix = "{0:0{pad}d}".format(prefix_value, pad=2)
    key = "{}/{}/{}".format(shard_prefix, date_str, shopper_id.lower())
    obj = s3_session.Object(s3_bucket, key)
    file_content = str(lz4.block.decompress(obj.get()["Body"].read()))
    return json.loads(file_content[2:-1].replace('\\\\"', '\\"').replace('\\\\/', '/').replace('\\\\\\\\/', '/'))
#     response = obj.get()
#     value = response["Body"].read()
#     content_type = response["ContentType"]
#     # Assume 'binary/octet-stream' is lz4 compressed.
#     if content_type in ("binary/octet-stream", "application/x-lz4"):
#         value = lz4.block.decompress(value)
#     return ujson.loads(value)

# From s3://gd-gxcoreservices-prod-shopperml-data/20221009/metadata.json
PRODUCTS_COLUMNS = [
      "product_type_id",
      "product_type_name",
      "product_name",
      "product_pnl_group_name",
      "product_pnl_category_name",
      "product_pnl_subline_name",
      "pf_id",
      "base_pf_id",
      "primary_product_flag",
      "auto_renewal_flag",
      "payment_profile_shopper_id",
      "domain_name",
      "original_list_price_amt",
      "domain_change_of_ownership_flag",
      "product_period_name",
      "product_period_qty",
      "private_label_id",
      "order_id",
      "original_order_product_free_trial_flag",
      "billing_sub_status_desc",
      "billing_status_name",
      "billing_attempt_sequence_name",
      "exclude_reason_desc",
      "create_date",
      "billing_date",
      "expiration_date",
      "last_renewal_date",
      "previous_expiration_date",
      "cancel_date",
      "catalog_price_group_desc",
      "source_table_name",
      "row_num"
    ]


def add_type(df):
    def normalize(s):
        return "" if s == None else s.lower()
    df["is_free"] = df.apply(lambda row: (normalize(row.billing_status_name) == "free")
                             | (normalize(row.billing_status_name) == "trial period")
                             | (normalize(row.billing_status_name) == "active")
                             & (normalize(row.exclude_reason_desc) == "add-on product"), axis=1)

    df["is_active"] = df.apply(lambda row: (normalize(row.billing_status_name) == "active")
                               & (normalize(row.billing_status_name) != "free")
                               & (normalize(row.billing_status_name) != "trial period")
                               & (normalize(row.exclude_reason_desc) != "add-on product"), axis=1)
    df["is_canceled"] = df.apply(lambda row: (normalize(row.billing_status_name) != "active") 
                               & (normalize(row.billing_status_name) != "free")
                               & (normalize(row.billing_status_name) != "trial period"), axis=1)
    return df

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
date = datetime(2022, 10, 2)
df = fv.get_historical_features(start_time=date, end_time=date + timedelta(days=1)).to_spark().toPandas()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
index = 0
for row in df.itertuples():
    print(row.shopper_id, 
          len(row.free_original_list_price_amt),
          len(row.active_original_list_price_amt),
          len(row.canceled_original_list_price_amt))
    if index > 500:
        break
    index += 1

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

100123631 0 0 0
109645318 0 0 0
110820935 1 0 0
135757547 0 0 0
168160710 0 0 0
173731310 0 0 0
176325052 0 0 0
19134977 0 0 0
191484145 1 0 0
198873818 0 0 1
204741285 0 0 0
207189890 0 0 0
210101103 1 0 0
210502647 0 0 0
214128000 0 0 0
226928522 0 0 2
230705421 0 0 0
23701929 0 0 0
242056715 0 0 0
249535714 1 0 0
249899913 0 0 0
259848070 1 0 0
265240351 1 0 0
2934187 0 0 0
303861215 0 0 0
323511604 2 0 1
332021786 1 0 0
333327478 0 0 0
341454110 0 0 0
35481312 0 0 0
357675272 0 0 0
35876480 0 0 0
36307141 0 0 0
363107483 0 0 0
383240493 1 0 0
383591748 0 0 0
408623233 0 0 0
417116729 0 0 0
434665059 0 0 0
44521670 0 0 0
445370493 0 0 0
448906312 1 0 0
449640049 0 0 1
451180561 2 0 0
454827747 3 0 0
45596322 0 0 0
459343001 0 0 0
460663182 1 0 0
461364513 0 0 0
46202212 2 0 0
46316084 4 0 0
466523744 0 0 0
468084990 0 0 0
469236866 0 0 0
490231293 1 0 0
49616000 0 0 0
4969686 1 0 0
519403094 1 0 0
525246884 1 0 0
526932324 0 0 0
529505839 0 0 0
534273843 1 0 0
541177162 0 0 0
541630

In [ ]:
#20221002 226928522: id not found shopperml
#         920309: multiple categories
#         162557657: empty tecton features
#         83011612: free email in shopperml but free website builder in tecton

In [17]:
shopper_id = "83011612"
for key, value in df[df.shopper_id == shopper_id].iloc[0].to_dict().items():
    print(f"{key}: {value}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

shopper_id: 83011612
active_original_list_price_amt: []
free_original_list_price_amt: [1.91]
canceled_original_list_price_amt: []
active_create_date: []
free_create_date: [1664668800]
canceled_create_date: []
active_expiration_date: []
free_expiration_date: [1667347200]
canceled_expiration_date: []
active_last_renewal_date: []
free_last_renewal_date: [1669075200]
canceled_last_renewal_date: []
active_cancel_date: []
free_cancel_date: []
canceled_cancel_date: []
active_billing_date: []
free_billing_date: [1667347200]
canceled_billing_date: []
active_product_name: []
free_product_name: ['Website Builder']
canceled_product_name: []
active_product_pnl_group_name: []
free_product_pnl_group_name: ['Presence and Commerce']
canceled_product_pnl_group_name: []
active_product_type_name: []
free_product_type_name: ['Website Builder wsb']
canceled_product_type_name: []
active_billing_attempt_sequence_name: []
free_billing_attempt_sequence_name: ['Active']
canceled_billing_attempt_sequence_name: []

In [18]:
df_shopperml = pd.DataFrame(get_data(date.strftime("%Y%m%d"), shopper_id)["uds_product_billing"], columns=PRODUCTS_COLUMNS)
df_shopperml = add_type(df_shopperml)
df_shopperml[["product_type_name", "original_list_price_amt", "billing_status_name", "exclude_reason_desc", "is_free", "is_active", "is_canceled"]]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

      product_type_name  original_list_price_amt billing_status_name  \
0   Website Builder wsb                   191.88              active   
1   Website Builder wsb                     0.00            canceled   
2   Website Builder wsb                     0.00            canceled   
3   Website Builder wsb                     0.00            canceled   
4   Website Builder wsb                     0.00            canceled   
5   Website Builder wsb                     0.00            canceled   
6   Website Builder wsb                     0.00            canceled   
7   Website Builder wsb                     0.00            canceled   
8   Website Builder wsb                     0.00            canceled   
9   Website Builder wsb                     0.00            canceled   
10  Website Builder wsb                     0.00            canceled   
11  Website Builder wsb                     0.00            canceled   
12  Website Builder wsb                     0.00            canc

In [19]:
df_shopperml

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    product_type_id    product_type_name  \
0               536  Website Builder wsb   
1               536  Website Builder wsb   
2               536  Website Builder wsb   
3               536  Website Builder wsb   
4               536  Website Builder wsb   
5               536  Website Builder wsb   
6               536  Website Builder wsb   
7               536  Website Builder wsb   
8               536  Website Builder wsb   
9               536  Website Builder wsb   
10              536  Website Builder wsb   
11              536  Website Builder wsb   
12              536  Website Builder wsb   
13              536  Website Builder wsb   
14              536  Website Builder wsb   
15              536  Website Builder wsb   
16              536  Website Builder wsb   
17              536  Website Builder wsb   
18              536  Website Builder wsb   
19              536  Website Builder wsb   
20              536  Website Builder wsb   
21               16       Email 